In [3]:
from datetime import datetime, timedelta
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

# FILTER 1
def get_codes():
    url = "https://www.mse.mk/en/stats/symbolhistory/ALK"
    with requests.Session() as session:
        response = session.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    dropdown = soup.find("select", id="Code")
    if not dropdown:
        return []
    return [
      option.text.strip()
      for option in dropdown.find_all("option")
      if not any(char.isdigit() for char in option.text) and not option.text.strip().startswith(('E'))
    ]
# FILTER 2
def get_last_update(code):
    path = f"{code}.csv"
    try:
        df = pd.read_csv(path)
        return pd.to_datetime(df['Date']).max()
    except (FileNotFoundError, pd.errors.EmptyDataError):
        return None
# FILTER 3
def fill_data(dataframe):
    numeric_cols = ['LastTradePrice', 'Max', 'Min', 'Avg. Price', '%chg.', 'Volume', 'Turnover in BEST', 'TotalTurnover']

    for col in numeric_cols:
        if col in dataframe.columns:
            dataframe[col] = dataframe[col].replace('', np.nan)  # Replace empty strings with NaN
            dataframe[col] = dataframe[col].replace({',': ''}, regex=True).astype(float)  # Remove commas and convert to float

    dataframe['Date'] = dataframe['Date'].ffill()

    for col in numeric_cols:
        if col in dataframe.columns:
            dataframe[col] = dataframe[col].fillna(dataframe[col].mean())

    for col in numeric_cols:
        if col in dataframe.columns:
            dataframe[col] = dataframe[col].apply(lambda x: f"{x:,.2f}")

def fetch_code(session, code, start_date, end_date):
    url = (
        f"https://www.mse.mk/en/stats/symbolhistory/{code}"
        f"?FromDate={start_date.strftime('%m/%d/%Y')}"
        f"&ToDate={end_date.strftime('%m/%d/%Y')}"
    )
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    tbody = soup.select_one('tbody')
    if not tbody:
        return []
    return [[cell.get_text(strip=True) for cell in row.find_all('td')] for row in tbody.find_all('tr')]

def update_code(session, code):
    current_date = datetime.now()
    last_update = get_last_update(code)
    all_data = []

    if last_update:
      start_date = (last_update + timedelta(days=1))
    else:
       start_date = current_date - timedelta(days=3650)

    while start_date <= current_date:
        year_end = datetime(start_date.year, 12, 31)
        end_date = min(year_end, current_date)
        data = fetch_code(session, code, start_date, end_date)
        all_data.extend(data)
        start_date = end_date + timedelta(days=1)

    if all_data:
        save_to_csv(code, all_data)

def save_to_csv(code, data):
    columns = ['Date', 'LastTradePrice', 'Max', 'Min', 'Avg. Price', '%chg.', 'Volume', 'Turnover in BEST', 'TotalTurnover']
    df = pd.DataFrame(data, columns=columns)

    fill_data(df)

    file_path = f"{code}.csv"
    df.to_csv(file_path, mode='a', header=not os.path.exists(file_path), index=False)

if __name__ == "__main__":
    codes = get_codes()
    with requests.Session() as session:
        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = {executor.submit(update_code, session, code): code for code in codes}
            for future in as_completed(futures):
                code = futures[future]
                try:
                    future.result()
                    print(f"{code} update complete.")
                except Exception as e:
                    print(f"Error updating {code}: {e}")


ALKB update complete.
ALK update complete.
AMEH update complete.
APTK update complete.
ADIN update complete.


<ipython-input-3-7bb4e224ae50>:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe[col] = dataframe[col].replace('', np.nan)  # Replace empty strings with NaN
<ipython-input-3-7bb4e224ae50>:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe[col] = dataframe[col].replace('', np.nan)  # Replace empty strings with NaN


ATPP update complete.
BIKF update complete.
BGOR update complete.
BANA update complete.
AUMK update complete.


<ipython-input-3-7bb4e224ae50>:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe[col] = dataframe[col].replace('', np.nan)  # Replace empty strings with NaN
<ipython-input-3-7bb4e224ae50>:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe[col] = dataframe[col].replace('', np.nan)  # Replace empty strings with NaN


BIM update complete.
BLTU update complete.
CKB update complete.
CKBKO update complete.


...

RZTK update complete.
RZUS update complete.
SBT update complete.
SDOM update complete.
SIL update complete.
SKON update complete.
NOSK update complete.
SKP update complete.
SLAV update complete.
SNBT update complete.
SNBTO update complete.
SPAZ update complete.
SOLN update complete.
SPAZP update complete.
SPOL update complete.
STIL update complete.
STBP update complete.
STB update complete.
SSPR update complete.
TAJM update complete.
TEAL update complete.
STOK update complete.
TEHN update complete.
TBKO update complete.
TIKV update complete.
TEL update complete.
TKPR update complete.
TETE update complete.
TKVS update complete.
TRPS update complete.
TRDB update complete.
TRUB update complete.
TSMP update complete.
TNB update complete.
TSZS update complete.
TTK update complete.
TTKO update complete.
USJE update complete.
VFPM update complete.
UNI update complete.
VARG update complete.
VSC update complete.
VTKS update complete.
VITA update complete.
VROS update complete.
ZAS update comple

<ipython-input-3-7bb4e224ae50>:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe[col] = dataframe[col].replace('', np.nan)  # Replace empty strings with NaN
<ipython-input-3-7bb4e224ae50>:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe[col] = dataframe[col].replace('', np.nan)  # Replace empty strings with NaN
<ipython-input-3-7bb4e224ae50>:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objec

ZILU update complete.


<ipython-input-3-7bb4e224ae50>:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe[col] = dataframe[col].replace('', np.nan)  # Replace empty strings with NaN
<ipython-input-3-7bb4e224ae50>:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe[col] = dataframe[col].replace('', np.nan)  # Replace empty strings with NaN


ZILUP update complete.
ZIMS update complete.
ZKAR update complete.
ZPKO update complete.
ZPOG update complete.
ZUAS update complete.


In [14]:
print(pd.read_csv('MZHE.csv'))

            Date LastTradePrice  Max  Min Avg. Price  %chg.  Volume  \
0     12/30/2014       3,133.00  NaN  NaN   3,133.00    0.0     0.0   
1     12/29/2014       3,133.00  NaN  NaN   3,133.00    0.0     0.0   
2     12/26/2014       3,133.00  NaN  NaN   3,133.00    0.0     0.0   
3     12/25/2014       3,133.00  NaN  NaN   3,133.00    0.0     0.0   
4     12/24/2014       3,133.00  NaN  NaN   3,133.00    0.0     0.0   
...          ...            ...  ...  ...        ...    ...     ...   
4893    1/9/2024       3,133.00  NaN  NaN   3,133.00    0.0     0.0   
4894    1/5/2024       3,133.00  NaN  NaN   3,133.00    0.0     0.0   
4895    1/4/2024       3,133.00  NaN  NaN   3,133.00    0.0     0.0   
4896    1/3/2024       3,133.00  NaN  NaN   3,133.00    0.0     0.0   
4897    1/2/2024       3,133.00  NaN  NaN   3,133.00    0.0     0.0   

      Turnover in BEST  TotalTurnover  
0                  0.0            0.0  
1                  0.0            0.0  
2                  0.0     